In [7]:
import pandas as pd
import numpy as np
import requests, bs4
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

In [44]:
url = 'http://openapi.airport.co.kr/service/rest/dailyExpectPassenger/dailyExpectPassenger?'
Encoding_key = 'S%2BsWqR2h%2F97Ss5FACtHfvTVQReGCG48oMZqwU%2BBwn9fdbo4qnWhEixkwthLjtJ5wouyDE%2FUnerh6rQSYLrdRZQ%3D%3D'

dict_airport = {'date':[], 'korean':[], 'foreigner':[]}
for m in [8, 9, 10]:
    for d in range(1, 32):
        if m < 10:
            date = '20190' + str(m)
        else: date = '2019' + str(m)

        if d < 10:
            date += '0' + str(d)
        else: date += str(d)

        domestic_url = url
        domestic_url += 'schDate=' + date + '&schAirport=CJU&schTof=D' + '&serviceKey=' + Encoding_key

        response = requests.get(domestic_url).text
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        domestic_count = 0
        for row in xmlobj.findAll('item'):
            for r in row.find_all('pct'):
                domestic_count += int(r.text)
        
        international_url = url
        international_url += 'schDate=' + date + '&schAirport=CJU&schTof=I' + '&serviceKey=' + Encoding_key

        response = requests.get(international_url).text
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        international_count = 0
        for row in xmlobj.findAll('item'):
            for r in row.find_all('pct'):
                international_count += int(r.text)
        
        if domestic_count != 0 and international_count != 0:
            dict_airport['date'].append(date)
            dict_airport['korean'].append(domestic_count)
            dict_airport['foreigner'].append(international_count)

airport = pd.DataFrame(dict_airport)
airport

,date,korean,foreigner
0,20190801,17431,2383
1,20190802,18687,2165
2,20190803,21698,3253
3,20190804,21878,2615
4,20190805,21924,1911
...,...,...,...
87,20191027,21644,3353
88,20191028,21220,2122
89,20191029,20414,3052
90,20191030,18073,2662


In [45]:
airport["date"] = pd.to_datetime(airport["date"])

airport["visitor"] = airport["korean"] + airport["foreigner"]
def get_previous_data(df):
    df["(t-3)"], df["(t-2)"], df["(t-1)"] = None, None, None
    for i in range(3, len(df)):
        df["(t-3)"][i] = int(df["visitor"][i-3])
        df["(t-2)"][i] = int(df["visitor"][i-2])
        df["(t-1)"][i] = int(df["visitor"][i-1])
    df.dropna(inplace=True)
    df = df.loc[31:76].reset_index(drop=True)
    return df
airport = get_previous_data(airport)

airport["visitor_3days"] = airport["(t-1)"] + airport["(t-2)"] + airport["(t-3)"]

airport

<ipython-input-45-447bca7abf2e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["(t-3)"][i] = int(df["visitor"][i-3])
<ipython-input-45-447bca7abf2e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["(t-2)"][i] = int(df["visitor"][i-2])
<ipython-input-45-447bca7abf2e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["(t-1)"][i] = int(df["visitor"][i-1])


,date,korean,foreigner,visitor,(t-3),(t-2),(t-1),visitor_3days
0,2019-09-01,21768,3262,25030,23304,23785,22271,69360
1,2019-09-02,20050,2097,22147,23785,22271,25030,71086
2,2019-09-03,16227,3067,19294,22271,25030,22147,69448
3,2019-09-04,12992,2079,15071,25030,22147,19294,66471
4,2019-09-05,13400,3046,16446,22147,19294,15071,56512
5,2019-09-06,16090,2232,18322,19294,15071,16446,50811
6,2019-09-07,6043,2865,8908,15071,16446,18322,49839
7,2019-09-08,14984,2797,17781,16446,18322,8908,43676
8,2019-09-09,12195,2013,14208,18322,8908,17781,45011
9,2019-09-10,13128,2417,15545,8908,17781,14208,40897


In [46]:
airport.to_csv('C:\\Users\\hskang\\Desktop\\Coding\\Python AI\\AI Competition 2nd\\Datas\\airport.csv')